In [354]:
# Use this block of code when working on Google Colab in order to save the files in Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

#path = "drive/MyDrive/Thesis/"

In [355]:
#%pip install transformers
#%pip install geopandas
#%pip install torch==1.13.1

In [356]:
import os
os.environ['USE_PYGEOS'] = '0'

from concurrent.futures import ThreadPoolExecutor, as_completed
import json

import matplotlib.pyplot as plt

from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
from PIL import Image, ImageDraw
import torch

import geopandas as gpd
from tqdm import tqdm
import numpy as np
import requests

import json

from scipy.signal import find_peaks
import pickle

In [357]:
# color palette to map each class to a RGB value
color_palette = [
    [128, 64, 128],  # 0: road - maroon
    [244, 35, 232],  # 1: sidewalk - pink
    [70, 70, 70],  # 2: building - dark gray
    [102, 102, 156],  # 3: wall - purple
    [190, 153, 153],  # 4: fence - light brown
    [153, 153, 153],  # 5: pole - gray
    [250, 170, 30],  # 6: traffic light - orange
    [220, 220, 0],  # 7: traffic sign - yellow
    [107, 142, 35],  # 8: vegetation - dark green
    [152, 251, 152],  # 9: terrain - light green
    [70, 130, 180],  # 10: sky - blue
    [220, 20, 60],  # 11: person - red
    [255, 0, 0],  # 12: rider - bright red
    [0, 0, 142],  # 13: car - dark blue
    [0, 0, 70],  # 14: truck - navy blue
    [0, 60, 100],  # 15: bus - dark teal
    [0, 80, 100],  # 16: train - dark green
    [0, 0, 230],  # 17: motorcycle - blue
    [119, 11, 32]  # 18: bicycle - dark red
]

#### The following functions were made just to visualise the results of some parts of the code and make sure that it was correctly working

In [358]:
import matplotlib.pyplot as plt

def visualize_results(image, segmentation):
    _, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Display the widened panorama image
    ax[0].imshow(image)
    ax[0].set_title("Widened Panorama Image")
    ax[0].axis("off")

    # Display the segmentation result
    ax[1].imshow(segmentation, cmap='jet', interpolation='nearest')
    ax[1].set_title("Segmentation Result")
    ax[1].axis("off")

    plt.tight_layout()
    plt.show()


In [359]:
def visualize_road_centres(segmentation, centres):
    palette_bytes = bytes([c for color in color_palette for c in color]) # concatenate rgb
    predict_np = segmentation.numpy()
    mask = Image.fromarray(predict_np.astype('uint8')).convert('P')
    mask.putpalette(palette_bytes)
    mask.load()
      
    draw = ImageDraw.Draw(mask)
      
    for centre in centres:
        draw.line((centre, 0, centre, mask.size[1]), width=4, fill=(0,255,0))
    
    mask.show()

In [370]:
from process_data import download_image, prepare_folders, get_models

def download_images_for_points(gdf, city, access_token, path=""):
    processor, model = get_models()
    prepare_folders(path, city)
    
    images_results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
  
        for _, row in gdf.iterrows():
            feature = row["feature"]
            geometry = row["geometry"]
            feature = json.loads(feature)
            futures.append(executor.submit(download_image, geometry, feature, city, access_token, processor, model, path))
 
        for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading images"):
            image_result = future.result()
            images_results.append(image_result)

    return gpd.GeoDataFrame(images_results, columns=["geometry", "GVI", "is_panoramic", "missing", "error"])

In [371]:
# Get the roadnetwork of a specific city using OpenStreetMap data
city = "Kampala, Uganda"

# Set access token for mapillary
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf"
path = ""

path_to_file="{}results/{}/data/points_with_features.gpkg".format(path, city)
gdf_features = gpd.read_file(path_to_file)
gdf_features = gdf_features.head(10)


In [372]:
results = download_images_for_points(gdf_features, city, access_token)

In [373]:
results

,geometry,GVI,is_panoramic,missing,error
0,POINT (32.59706 0.33938),0.000000,None,True,False
1,POINT (32.59995 0.33730),0.000000,None,True,False
2,POINT (32.59793 0.33903),0.000000,None,True,False
3,POINT (32.59969 0.33767),0.000000,None,True,False
4,POINT (32.59872 0.33859),0.000000,None,True,False
5,POINT (32.59941 0.33802),0.000000,None,True,False
6,POINT (32.59714 0.33946),0.336839,False,False,False
7,POINT (32.59753 0.33924),0.232132,False,False,False
8,POINT (32.59908 0.33833),0.206590,False,False,False
9,POINT (32.59832 0.33881),0.186828,False,False,False
